## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [ ]:
from datasets import load_dataset, get_dataset_split_names

dataset_name = "rotten_tomatoes"

# Questo è utile per sapere in anticipo quali split sono presenti
try:
    available_splits = get_dataset_split_names(dataset_name)
    print(f"Split disponibili per il dataset '{dataset_name}': {available_splits}")
except Exception as e:
    print(f"Errore nel recuperare i nomi degli split per '{dataset_name}': {e}")
    print("Potrebbe essere necessario caricare il dataset prima di poter ispezionare gli split, o il dataset potrebbe non avere split predefiniti.")

print("-" * 30)

print(f"Caricamento del dataset '{dataset_name}'...")
dataset = load_dataset(dataset_name)
print("Dataset caricato con successo!")

print("-" * 30)

print("Esplorazione del dataset:")

print(f"\nSplit caricati nel DatasetDict: {dataset.keys()}")

# Iterare su ogni split e mostrare alcune informazioni
for split_name in dataset.keys():
    print(f"\n--- Split: '{split_name}' ---")
    current_split = dataset[split_name]

    print(f"Numero di esempi nello split '{split_name}': {len(current_split)}")

    # Mostra la struttura delle features (colonne)
    print(f"Features dello split '{split_name}':")
    print(current_split.features)

    # Mostra i primi 3 esempi di questo split
    print(f"Primi 3 esempi dallo split '{split_name}':")
    for i in range(min(3, len(current_split))): 
        print(f"  Esempio {i}: {current_split[i]}")

    if 'label' in current_split.features:
        label_feature = current_split.features['label']
        if hasattr(label_feature, 'names'): 
            print(f"  Nomi delle etichette (labels) in '{split_name}': {label_feature.names}")
        else:
            print(f"  Tipo di etichetta (label) in '{split_name}': {label_feature.dtype}")

Split disponibili per il dataset 'rotten_tomatoes': ['train', 'validation', 'test']
------------------------------
Caricamento del dataset 'rotten_tomatoes'...
Dataset caricato con successo!
------------------------------
Esplorazione del dataset:

Split caricati nel DatasetDict: dict_keys(['train', 'validation', 'test'])

--- Split: 'train' ---
Numero di esempi nello split 'train': 8530
Features dello split 'train':
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
Primi 3 esempi dallo split 'train':
  Esempio 0: {'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
  Esempio 1: {'text': 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien

#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [ ]:
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset 

# 'uncased' significa che il modello non distingue tra maiuscole e minuscole 
model_name = "distilbert-base-uncased"

# Caricare il tokenizer pre-addestrato
print(f"Caricamento del tokenizer per '{model_name}'...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer caricato con successo!")

# Caricare il modello pre-addestrato
# AutoModel per caricare il modello base.
print(f"Caricamento del modello '{model_name}'...")
model = AutoModel.from_pretrained(model_name)
print("Modello caricato con successo!")

print("-" * 30)


try:
    dataset = load_dataset("rotten_tomatoes")
    sample_texts = [
        dataset['train'][0]['text'], # Esempio positivo dal training set
        dataset['train'][1]['text'], # Esempio negativo dal training set
        "This movie was absolutely fantastic! Highly recommended.", # Un esempio positivo
        "I regret wasting my time on this dreadful film." # Un esempio negativo
    ]
except NameError:
    print("Dataset 'rotten_tomatoes' non trovato. Si prega di caricarlo prima di eseguire questa sezione o di fornire esempi manuali.")
    sample_texts = [
        "This movie was a cinematic masterpiece.",
        "Absolutely the worst film I've seen this year.",
        "A truly enjoyable experience.",
        "Boring and uninspired."
    ]

print("Test di tokenizzazione e passaggio al modello con i seguenti esempi:")
for i, text in enumerate(sample_texts):
    print(f"  Esempio {i+1}: \"{text}\"")

print("-" * 30)

# Tokenizzare i campioni
# Il tokenizer converte il testo in input_ids, attention_mask, e (opzionalmente) token_type_ids.
# return_tensors='pt' restituisce i tensori PyTorch, necessari per il modello.
# truncation=True tronca le sequenze più lunghe della massima lunghezza supportata dal modello (512 per BERT/DistilBERT).
# padding=True aggiunge padding alle sequenze più corte per raggiungere la massima lunghezza o la lunghezza della sequenza più lunga nel batch.
print("Tokenizzazione degli esempi...")
tokenized_inputs = tokenizer(sample_texts,
                             padding=True,
                             truncation=True,
                             return_tensors='pt') # 'pt' for PyTorch tensors

print("\nOutput del tokenizer per il primo esempio (sample_texts[0]):")
print("Input IDs:", tokenized_inputs['input_ids'][0])
print("Attention Mask:", tokenized_inputs['attention_mask'][0])

print("\nDecodifica dei primi Input IDs per visualizzare i token:")
decoded_tokens = tokenizer.decode(tokenized_inputs['input_ids'][0])
print(f"Testo originale: \"{sample_texts[0]}\"")
print(f"Token decodificati: \"{decoded_tokens}\"")

print("-" * 30)

# Passare i token attraverso il modello
print("Passaggio dei token attraverso il modello DistilBERT...")
with_no_grad = True # Per risparmiare memoria durante l'inferenza, non calcoliamo i gradienti

if with_no_grad:
    import torch
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
else:
    outputs = model(**tokenized_inputs)

print("Output del modello generati con successo!")

# Esaminare gli output del modello
# I modelli AutoModel restituiscono un oggetto simile a un dizionario.
# Per DistilBERT (AutoModel), l'output principale è `last_hidden_state`.
# Questo è l'output dell'ultimo strato del Transformer per ogni token.
print("\nStruttura degli output del modello:")
print(outputs)

print("\nShape di 'last_hidden_state' (batch_size, sequence_length, hidden_size):")
print(outputs.last_hidden_state.shape) # Esempio: torch.Size([4, 50, 768])
# 4 è il numero di esempi, 50 è la lunghezza massima della sequenza dopo padding,
# 768 è la dimensione dell'embedding nascosto di DistilBERT.

print("\nOutput nascosto (embedding) per il token [CLS] del primo esempio:")
# Il token [CLS] (Class Label) è il primo token (indice 0) nella sequenza.
# Per compiti di classificazione di sequenza, l'embedding di [CLS] dell'ultimo strato
# è spesso usato come rappresentazione dell'intera frase.
print(outputs.last_hidden_state[0, 0, :10]) # Mostra i primi 10 valori dell'embedding

print("\nTask completato: Tokenizer e Modello caricati, esempi tokenizzati e passati al modello.")

Caricamento del tokenizer per 'distilbert-base-uncased'...
Tokenizer caricato con successo!
Caricamento del modello 'distilbert-base-uncased'...
Modello caricato con successo!
------------------------------
Test di tokenizzazione e passaggio al modello con i seguenti esempi:
  Esempio 1: "the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal ."
  Esempio 2: "the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth ."
  Esempio 3: "This movie was absolutely fantastic! Highly recommended."
  Esempio 4: "I regret wasting my time on this dreadful film."
------------------------------
Tokenizzazione degli esempi...

Output del tokenizer per il primo esempio (sample_texts[0]):
Input IDs: tensor([  101,  199

#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [3]:
import numpy as np
from datasets import load_dataset
from transformers import pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tqdm.auto import tqdm # Per visualizzare una barra di avanzamento durante l'estrazione delle features

In [4]:
# Caricare il dataset (se non lo hai già fatto)
dataset_name = "rotten_tomatoes"
try:
    dataset = load_dataset(dataset_name)
    print(f"Dataset '{dataset_name}' caricato.")
except Exception as e:
    print(f"Errore nel caricare il dataset: {e}")
    print("Assicurati di aver installato la libreria 'datasets' e di essere connesso a internet.")
    exit()

Dataset 'rotten_tomatoes' caricato.


In [5]:
model_name = "distilbert-base-uncased"

# Inizializzare la pipeline di feature extraction

feature_extractor_pipeline = pipeline(
    "feature-extraction",
    model=model_name,
    tokenizer=model_name,
    device=0 if np.array(0).dtype == np.int_ else -1 # Esempio per rilevare se è disponibile una GPU
)
print(f"\nPipeline di feature extraction con '{model_name}' pronta.")

# Funzione per estrarre le features (embedding del CLS token)
def extract_cls_features(batch):
    
    features = feature_extractor_pipeline(batch['text'])

    # Estraggo l'embedding del CLS token (il primo vettore [0][0]) per ogni esempio nel batch.
    # Converto da lista a numpy array per comodità.

    cls_embeddings = [np.array(f[0][0]) for f in features]
    return {'features': cls_embeddings, 'labels': batch['label']}

print("\nEstrazione delle features dai dataset split (potrebbe richiedere tempo)...")

# Applica la funzione di estrazione features a tutti gli split del dataset
# Uso map con batch=True per processare più esempi contemporaneamente, migliorando l'efficienza.
# tqdm.auto è usato per visualizzare il progresso.
extracted_dataset = dataset.map(
    extract_cls_features,
    batched=True,
    batch_size=16, 
    remove_columns=['text'], 
    desc="Extracting features"
)

# Verifica la struttura del dataset con le nuove features
print("\nStruttura del dataset dopo l'estrazione delle features:")
print(extracted_dataset)
print(f"Esempio di feature estratta (primo elemento dello split train): {extracted_dataset['train'][0]['features'][:5]}...") # Mostra i primi 5 elementi dell'embedding
print(f"Dimensione dell'embedding (hidden_size di DistilBERT): {len(extracted_dataset['train'][0]['features'])}") 

print("-" * 30)

Device set to use cuda:0



Pipeline di feature extraction con 'distilbert-base-uncased' pronta.

Estrazione delle features dai dataset split (potrebbe richiedere tempo)...


Extracting features:   0%|          | 0/8530 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Extracting features:   0%|          | 0/1066 [00:00<?, ? examples/s]

Extracting features:   0%|          | 0/1066 [00:00<?, ? examples/s]


Struttura del dataset dopo l'estrazione delle features:
DatasetDict({
    train: Dataset({
        features: ['label', 'features', 'labels'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['label', 'features', 'labels'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['label', 'features', 'labels'],
        num_rows: 1066
    })
})
Esempio di feature estratta (primo elemento dello split train): [-0.033173568546772, -0.01680893637239933, 0.01941203698515892, -0.025717880576848984, -0.1379668116569519]...
Dimensione dell'embedding (hidden_size di DistilBERT): 768
------------------------------


In [6]:
# Addestrare un classificatore (SVM da Scikit-learn)
print("\nAddestramento del classificatore SVM...")

# Preparare i dati per l'SVM
X_train = np.array(extracted_dataset['train']['features'])
y_train = np.array(extracted_dataset['train']['labels'])

X_val = np.array(extracted_dataset['validation']['features'])
y_val = np.array(extracted_dataset['validation']['labels'])

X_test = np.array(extracted_dataset['test']['features'])
y_test = np.array(extracted_dataset['test']['labels'])

print(f"Dimensioni dei dati di training: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Dimensioni dei dati di validation: X_val={X_val.shape}, y_val={y_val.shape}")
print(f"Dimensioni dei dati di test: X_test={X_test.shape}, y_test={y_test.shape}")


# Inizializzare e addestrare la SVM

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_train, y_train)

print("Classificatore SVM addestrato con successo!")

print("-" * 30)


Addestramento del classificatore SVM...
Dimensioni dei dati di training: X_train=(8530, 768), y_train=(8530,)
Dimensioni dei dati di validation: X_val=(1066, 768), y_val=(1066,)
Dimensioni dei dati di test: X_test=(1066, 768), y_test=(1066,)
Classificatore SVM addestrato con successo!
------------------------------


In [7]:
# Valutare le prestazioni sul validation e test splits
print("\nValutazione delle prestazioni...")

# Previsioni sul set di validazione
y_val_pred = svm_classifier.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nAccuracy sul set di validazione: {val_accuracy:.4f}")
print("Report di classificazione sul set di validazione:")
print(classification_report(y_val, y_val_pred, target_names=['negative', 'positive']))

# Previsioni sul set di test
y_test_pred = svm_classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\nAccuracy sul set di test: {test_accuracy:.4f}")
print("Report di classificazione sul set di test:")
print(classification_report(y_test, y_test_pred, target_names=['negative', 'positive']))

print("\nStable Baseline calcolata!")


Valutazione delle prestazioni...

Accuracy sul set di validazione: 0.8189
Report di classificazione sul set di validazione:
              precision    recall  f1-score   support

    negative       0.80      0.85      0.82       533
    positive       0.84      0.79      0.81       533

    accuracy                           0.82      1066
   macro avg       0.82      0.82      0.82      1066
weighted avg       0.82      0.82      0.82      1066


Accuracy sul set di test: 0.8068
Report di classificazione sul set di test:
              precision    recall  f1-score   support

    negative       0.80      0.82      0.81       533
    positive       0.82      0.79      0.80       533

    accuracy                           0.81      1066
   macro avg       0.81      0.81      0.81      1066
weighted avg       0.81      0.81      0.81      1066


Stable Baseline calcolata!


-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [8]:
import numpy as np
from datasets import load_dataset, DatasetDict # Importa DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report, accuracy_score

In [9]:
# --- Caricare il dataset (se non l'hai già in memoria) ---
dataset_name = 'rotten_tomatoes'
try:
    dataset = load_dataset(dataset_name)
    print(f"Dataset '{dataset_name}' caricato.")
    print("Struttura iniziale del dataset:")
    print(dataset)
    print("Esempio di un elemento prima della tokenizzazione:")
    print(dataset['train'][0])
except Exception as e:
    print(f"Errore nel caricare il dataset: {e}")
    print("Assicurati di aver installato la libreria 'datasets' e di essere connesso a internet.")
    exit()

Dataset 'rotten_tomatoes' caricato.
Struttura iniziale del dataset:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
Esempio di un elemento prima della tokenizzazione:
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}


In [10]:
# --- Caricare il tokenizer di DistilBERT ---
model_name = 'distilbert-base-uncased'
print(f"\nCaricamento del tokenizer '{model_name}'...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer caricato con successo!")


Caricamento del tokenizer 'distilbert-base-uncased'...
Tokenizer caricato con successo!


In [11]:
# --- Definire la funzione di tokenizzazione ---
def tokenize_function(examples):
    # Il tokenizer processa il campo 'text'.
    # padding='max_length' padderà tutte le sequenze alla lunghezza massima del modello (512 per DistilBERT)
    # truncation=True tronca le sequenze più lunghe della lunghezza massima del modello
    return tokenizer(examples['text'], padding='max_length', truncation=True)


In [12]:
# --- Applicare la funzione di tokenizzazione a tutti gli split del dataset ---
print("\nApplicazione della funzione di tokenizzazione al dataset (potrebbe richiedere tempo)...")
# Usiamo batched=True per processare più esempi contemporaneamente, il che è più efficiente.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

print("\nDataset tokenizzato e trasformato.")
print("Struttura del dataset dopo la tokenizzazione:")
print(tokenized_datasets)



Applicazione della funzione di tokenizzazione al dataset (potrebbe richiedere tempo)...


Map:   0%|          | 0/1066 [00:00<?, ? examples/s]


Dataset tokenizzato e trasformato.
Struttura del dataset dopo la tokenizzazione:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})


In [13]:
# --- Verificare che i nuovi dataset restituiscano le colonne desiderate ---
print("\nVerifica delle colonne del primo elemento dello split 'train':")
first_train_element = tokenized_datasets['train'][0]
print(f"Contiene 'text': {'text' in first_train_element}")
print(f"Contiene 'label': {'label' in first_train_element}")
print(f"Contiene 'input_ids': {'input_ids' in first_train_element}")
print(f"Contiene 'attention_mask': {'attention_mask' in first_train_element}")

# Esempio completo di un elemento tokenizzato
print("\nEsempio del primo elemento tokenizzato dello split 'train':")
print(first_train_element)
# Decodificare gli input_ids per vedere il testo originale ricostruito (utile per debug)
print(f"\nTesto decodificato (dai primi 50 input_ids): {tokenizer.decode(first_train_element['input_ids'][:50])}...")


Verifica delle colonne del primo elemento dello split 'train':
Contiene 'text': True
Contiene 'label': True
Contiene 'input_ids': True
Contiene 'attention_mask': True

Esempio del primo elemento tokenizzato dello split 'train':
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1, 'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [14]:
# --- Caricare il modello pre-addestrato con una testa di classificazione ---
# Sto usando DistilBERT per la classificazione di sequenze.

num_labels = 2 # Per sentiment analysis: negativo/positivo

print(f"\nCaricamento del modello '{model_name}' per la classificazione di sequenze (num_labels={num_labels})...")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

print("Model architecture:")
print(model)


Caricamento del modello 'distilbert-base-uncased' per la classificazione di sequenze (num_labels=2)...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model architecture:
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=

#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [15]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [16]:
# --- Instanziare un DataCollatorWithPadding ---
# Questo Data Collator padderà dinamicamente le sequenze nel batch alla lunghezza massima del batch.
print("\nInizializzazione del DataCollatorWithPadding...")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("DataCollatorWithPadding inizializzato.")



Inizializzazione del DataCollatorWithPadding...
DataCollatorWithPadding inizializzato.


In [17]:
# --- Scrivere una funzione di valutazione (compute_metrics) ---
def compute_metrics(eval_pred):
    """
    Funzione per calcolare le metriche di valutazione.
    Prende un oggetto EvalPrediction (tuple di logits e labels) e restituisce un dizionario di metriche.
    """
    logits, labels = eval_pred # eval_pred è un tuple (logits, labels)

    # Converte i logits in previsioni di classe (l'indice con il valore più alto)
    predictions = np.argmax(logits, axis=-1)

    # Calcola l'accuratezza
    accuracy = accuracy_score(labels, predictions)

    # Calcola precisione, recall e F1-score per ogni classe e il macro average
    # labels=[0, 1] per assicurarsi che tutte le classi siano considerate
    # average='binary' per il f1, precision, recall per la classe positiva (1)
    # average='macro' per calcolare la media non pesata delle metriche per ogni classe
    # average='weighted' per calcolare la media pesata dal numero di istanze per ogni classe
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, labels=[0, 1])

    # Crea un dizionario con tutte le metriche
    metrics = {
        "accuracy": accuracy,
        "positive_precision": precision[1], # Precisione per la classe positiva (1)
        "positive_recall": recall[1],       # Recall per la classe positiva (1)
        "positive_f1": f1[1],               # F1-score per la classe positiva (1)
        "negative_precision": precision[0], # Precisione per la classe negativa (0)
        "negative_recall": recall[0],       # Recall per la classe negativa (0)
        "negative_f1": f1[0],               # F1-score per la classe negativa (0)
        "macro_f1": np.mean([f1[0], f1[1]]) # Macro F1-score
    }
    return metrics

In [18]:
# --- Instanziare un oggetto TrainingArguments ---
print("\nDefinizione degli argomenti di training...")
training_args = TrainingArguments(
    output_dir='./results',          # Directory dove verranno salvati i checkpoint del modello e i risultati
    eval_strategy='epoch',           # Esegue la valutazione alla fine di ogni epoca
    num_train_epochs=3,              # Numero di epoche di training
    per_device_train_batch_size=16,  # Dimensione del batch per training per GPU/CPU
    per_device_eval_batch_size=64,   # Dimensione del batch per valutazione per GPU/CPU
    warmup_steps=500,                # Numero di step di "warmup" per lo scheduler del learning rate
    weight_decay=0.01,               # Peso per la regolarizzazione L2 (utile per prevenire overfitting)
    logging_dir='./logs',            # Directory per i log di TensorBoard
    logging_steps=500,               # Registra i log ogni 500 step
    save_strategy="epoch",           # Salva il modello alla fine di ogni epoca
    load_best_model_at_end=True,     # Carica il miglior modello (basato su eval_strategy) alla fine del training
    metric_for_best_model="accuracy",# Specifica la metrica per determinare il "miglior" modello
    report_to='none',                # Non inviare log a servizi esterni (es. wandb, mlflow)
    fp16=False,                      # Imposta a True se hai una GPU compatibile per l'addestramento in mixed precision
)
print("TrainingArguments definiti.")


Definizione degli argomenti di training...
TrainingArguments definiti.


In [19]:
# --- Instanziare un Trainer ---
print("\nInizializzazione del Trainer...")
trainer = Trainer(
    model=model,                         # Il modello DistilBERT per la classificazione
    args=training_args,                  # Gli argomenti di training definiti
    train_dataset=tokenized_datasets['train'], # Il dataset di training tokenizzato
    eval_dataset=tokenized_datasets['validation'], # Il dataset di validazione tokenizzato
    data_collator=data_collator,         # Il data collator per la creazione dei batch
    compute_metrics=compute_metrics,     # La funzione per calcolare le metriche di valutazione
    tokenizer=tokenizer                  # Passiamo il tokenizer al Trainer per usi interni (es. logging)
)
print("Trainer inizializzato.")



Inizializzazione del Trainer...
Trainer inizializzato.


/tmp/ipykernel_1257995/633860319.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# --- Chiamare trainer.train() e trainer.evaluate() ---
print("\nAvvio del fine-tuning (potrebbe richiedere tempo)...")
trainer.train()
print("\nFine-tuning completato.")

print("\nValutazione delle prestazioni sul set di test...")
# Esegui la valutazione sul set di test, che non è stato usato durante l'addestramento
test_results = trainer.evaluate(tokenized_datasets['test'])
print("\nRisultati sul set di test dopo il fine-tuning:")
print(test_results)

# Per un report di classificazione più dettagliato sul set di test:
# Ottieni le previsioni raw (logits) sul set di test
predictions = trainer.predict(tokenized_datasets['test'])
# Converti i logits in etichette predette
predicted_labels = np.argmax(predictions.predictions, axis=-1)
# Ottieni le etichette reali
true_labels = tokenized_datasets['test']['label']

print("\nReport di classificazione dettagliato sul set di test:")
print(classification_report(true_labels, predicted_labels, target_names=['negative', 'positive']))

print("\nComparazione con la baseline (accuracy del test set):")
print(f"- Baseline (SVM su feature estratte): circa 0.8068")
print(f"- Fine-tuned DistilBERT: {test_results['eval_accuracy']:.4f}")


Avvio del fine-tuning (potrebbe richiedere tempo)...


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Positive Precision,Positive Recall,Positive F1,Negative Precision,Negative Recall,Negative F1,Macro F1
1,No log,0.399236,0.826454,0.779743,0.909944,0.839827,0.891892,0.742964,0.810645,0.825236
2,0.409000,0.381858,0.826454,0.812950,0.848030,0.830119,0.841176,0.804878,0.822627,0.826373
3,0.409000,0.454157,0.849906,0.836036,0.870544,0.852941,0.864971,0.829268,0.846743,0.849842


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Fine-tuning completato.

Valutazione delle prestazioni sul set di test...


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Risultati sul set di test dopo il fine-tuning:
{'eval_loss': 0.5289661884307861, 'eval_accuracy': 0.8311444652908068, 'eval_positive_precision': 0.8203266787658802, 'eval_positive_recall': 0.8480300187617261, 'eval_positive_f1': 0.8339483394833949, 'eval_negative_precision': 0.8427184466019417, 'eval_negative_recall': 0.8142589118198874, 'eval_negative_f1': 0.8282442748091603, 'eval_macro_f1': 0.8310963071462776, 'eval_runtime': 19.9007, 'eval_samples_per_second': 53.566, 'eval_steps_per_second': 0.452, 'epoch': 3.0}


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Report di classificazione dettagliato sul set di test:
              precision    recall  f1-score   support

    negative       0.84      0.81      0.83       533
    positive       0.82      0.85      0.83       533

    accuracy                           0.83      1066
   macro avg       0.83      0.83      0.83      1066
weighted avg       0.83      0.83      0.83      1066


Comparazione con la baseline (accuracy del test set):
- Baseline (SVM su feature estratte): circa 0.8068
- Fine-tuned DistilBERT: 0.8311


-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [21]:
from peft import LoraConfig, get_peft_model, TaskType

In [22]:
dataset_name = 'rotten_tomatoes'
model_name = 'distilbert-base-uncased' 
num_labels = 2 

print(f"Caricamento del dataset '{dataset_name}'...")
dataset = load_dataset(dataset_name)
print("Dataset caricato con successo!")

print(f"\nCaricamento del tokenizer '{model_name}'...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer caricato con successo!")

# Funzione di tokenizzazione (come nell'Esercizio 2.1)
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

print("\nApplicazione della funzione di tokenizzazione al dataset...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print("Dataset tokenizzato e trasformato.")

Caricamento del dataset 'rotten_tomatoes'...
Dataset caricato con successo!

Caricamento del tokenizer 'distilbert-base-uncased'...
Tokenizer caricato con successo!

Applicazione della funzione di tokenizzazione al dataset...


Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset tokenizzato e trasformato.


In [23]:
# --- Caricamento del modello base (come nell'Esercizio 2.2) ---
print(f"\nCaricamento del modello '{model_name}' per la classificazione di sequenze (num_labels={num_labels})...")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
print("Modello base caricato con successo!")


Caricamento del modello 'distilbert-base-uncased' per la classificazione di sequenze (num_labels=2)...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modello base caricato con successo!


In [24]:
# --- Configurazione e applicazione di PEFT (LoRA) (Nuovo per l'Esercizio 3.1) ---
print("\nConfigurazione e applicazione di PEFT (LoRA)...")
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"], # Moduli di attenzione in DistilBERT
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(model, peft_config)
print("Modello trasformato in PEFT (LoRA) con successo.")
model.print_trainable_parameters() # Mostra quanti parametri sono ora addestrabili


Configurazione e applicazione di PEFT (LoRA)...
Modello trasformato in PEFT (LoRA) con successo.
trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [25]:
# --- Instanziare un DataCollatorWithPadding ---
print("\nInizializzazione del DataCollatorWithPadding...")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("DataCollatorWithPadding inizializzato.")



Inizializzazione del DataCollatorWithPadding...
DataCollatorWithPadding inizializzato.


Tutto il resto è già stato definito nel punto precendente.

In [26]:
# --- Avviare l'addestramento e la valutazione ---
print("\nAvvio del fine-tuning con PEFT (potrebbe richiedere meno tempo)...")
trainer.train()
print("\nFine-tuning con PEFT completato.")

print("\nValutazione delle prestazioni sul set di test con il modello PEFT...")
test_results_peft = trainer.evaluate(tokenized_datasets['test'])
print("\nRisultati sul set di test dopo il fine-tuning con PEFT:")
print(test_results_peft)

# Per un report di classificazione più dettagliato sul set di test:
predictions_peft = trainer.predict(tokenized_datasets['test'])
predicted_labels_peft = np.argmax(predictions_peft.predictions, axis=-1)
true_labels_peft = tokenized_datasets['test']['label']

print("\nReport di classificazione dettagliato sul set di test (PEFT):")
print(classification_report(true_labels_peft, predicted_labels_peft, target_names=['negative', 'positive']))

print("\nComparazione con le prestazioni precedenti (accuracy del test set):")
print(f"- Baseline (SVM su feature estratte): circa 0.8068")
# Usa l'accuracy del tuo fine-tuning precedente per il confronto diretto
print(f"- Fine-tuned DistilBERT (full): Il tuo risultato precedente (circa 0.8396 o 0.8321)")
print(f"- Fine-tuned DistilBERT (PEFT LoRA): {test_results_peft['eval_accuracy']:.4f}")



Avvio del fine-tuning con PEFT (potrebbe richiedere meno tempo)...


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Positive Precision,Positive Recall,Positive F1,Negative Precision,Negative Recall,Negative F1,Macro F1
1,No log,0.604171,0.843340,0.841418,0.846154,0.843779,0.845283,0.840525,0.842897,0.843338
2,0.072000,0.549464,0.848968,0.838182,0.864916,0.851339,0.860465,0.833021,0.846520,0.848930
3,0.072000,0.758060,0.851782,0.841530,0.866792,0.853974,0.862669,0.836773,0.849524,0.851749


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Fine-tuning con PEFT completato.

Valutazione delle prestazioni sul set di test con il modello PEFT...


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Risultati sul set di test dopo il fine-tuning con PEFT:
{'eval_loss': 0.8086183667182922, 'eval_accuracy': 0.8348968105065666, 'eval_positive_precision': 0.8275229357798165, 'eval_positive_recall': 0.8461538461538461, 'eval_positive_f1': 0.8367346938775511, 'eval_negative_precision': 0.8426103646833013, 'eval_negative_recall': 0.8236397748592871, 'eval_negative_f1': 0.8330170777988615, 'eval_macro_f1': 0.8348758858382063, 'eval_runtime': 20.0158, 'eval_samples_per_second': 53.258, 'eval_steps_per_second': 0.45, 'epoch': 3.0}


/home/mdonadio/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Report di classificazione dettagliato sul set di test (PEFT):
              precision    recall  f1-score   support

    negative       0.84      0.82      0.83       533
    positive       0.83      0.85      0.84       533

    accuracy                           0.83      1066
   macro avg       0.84      0.83      0.83      1066
weighted avg       0.84      0.83      0.83      1066


Comparazione con le prestazioni precedenti (accuracy del test set):
- Baseline (SVM su feature estratte): circa 0.8068
- Fine-tuned DistilBERT (full): Il tuo risultato precedente (circa 0.8396 o 0.8321)
- Fine-tuned DistilBERT (PEFT LoRA): 0.8349


## Commento sui Risultati
Questi risultati sono molto positivi e confermano il valore delle tecniche *** PEFT ***.

### Prestazioni Solide del Modello PEFT: 
Il modello fine-tuned con PEFT (LoRA) raggiunge un'accuratezza di circa 83.49%. 

### Efficienza con Prestazioni Competing: 
Ciò che rende questi risultati particolarmente impressionanti è che, pur addestrando solo una minuscola frazione dei parametri totali del modello (circa l'1%), le prestazioni sono quasi identiche a quelle del fine-tuning completo. La differenza rispetto al miglior risultato di fine-tuning completo (0.8396) è minima, di soli 0.0047.

### Accuratezza e Metriche Bilanciate: 
Il modello continua a mostrare performance equilibrate tra le classi "negative" e "positive", con precisione, recall e F1-score che si aggirano intorno a 0.83−0.85. Questo significa che è altrettanto efficace nell'identificare entrambi i tipi di sentiment.

In sintesi, si è dimostrato con successo che le tecniche di fine-tuning efficiente come PEFT (LoRA) permettono di ottenere prestazioni di alto livello, quasi indistinguibili da quelle del fine-tuning completo, ma con un costo computazionale e di memoria drasticamente ridotto. Questo è un vantaggio enorme in scenari reali, dove le risorse sono spesso limitate.



Perchè ci sono tanti **`UserWarning`**?

L'output del modello è così "compatto" che non ha una dimensione aggiuntiva per essere direttamente "impilato". PyTorch risolve questo problema automaticamente facendo "unsqueeze" (aggiungendo una dimensione di grandezza 1) a questi scalari e poi raggruppandoli in un vettore.
